In [34]:
import pymongo

myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient["douyin"]
users = mydb.users
user = users.find_one()
user

{'_id': ObjectId('5de09fbf1b9be34699ef23f6'),
 'status_code': 0,
 'user': {'is_activity_user': False,
  'followers_detail': [{'app_name': 'aweme',
    'name': '抖音',
    'icon': 'http://p3.pstatp.com/origin/50ec00079b64de2050dc',
    'fans_count': 83552,
    'open_url': 'snssdk1128://user/profile/1077159999315757?',
    'apple_id': '1142110895',
    'download_url': 'https://d.douyin.com/JsvN/',
    'package_name': 'com.ss.android.ugc.aweme'},
   {'name': '头条',
    'icon': 'http://p3.pstatp.com/origin/50ed00079a1b6b8d1fb1',
    'fans_count': 30,
    'open_url': 'snssdk143://profile?uid=69840027238',
    'apple_id': '529092160',
    'download_url': 'https://d.toutiao.com/YjjY/',
    'package_name': 'com.ss.android.article.news',
    'app_name': 'news_article'},
   {'icon': 'http://p3.pstatp.com/origin/551900041a7e00ec86ca',
    'fans_count': 0,
    'open_url': 'snssdk1112://profile?id=1077159999315757',
    'apple_id': '1086047750',
    'download_url': 'http://d.huoshanzhibo.com/eFvB/',
 

In [35]:
logging_sample = mydb.logging.find_one()
logging_sample



In [11]:
aggs = [
    {
        "$group":{
            "_id": "$user.gender",
            "count": {"$sum":1},
            "avg_dongtai_count": {"$avg":"$user.dongtai_count"}
        }
    }
]

result = users.aggregate(pipeline=aggs)

In [12]:
for i in result:
    print(i)

在user属性中发现了is_star字段，对其进行聚合统计的结果如下，虽然不知道背后的设置标准，但是我们可以初步使用此字段进行“大V&粉丝”之间的区分

In [13]:
match = {'$match': {'user.is_star': {'$ne': None},
                     'user.gender': {'$ne': None},
                     'user.follower_count': {'$gt': 100000},
                     'user.following_count': {'$ne': None},
                     'user.total_favorited': {'$ne': None}}}  
            
group = {"$group":{
            "_id": {"is_star":"$user.is_star"},
            "count": {"$sum":1},
            "avg_follower_count": {"$avg":"$user.follower_count"},
            "max_follower_count": {"$max":"$user.follower_count"},
            "min_follower_count": {"$min":"$user.follower_count"},
            "avg_following_count": {"$avg":"$user.following_count"},
            "max_following_count": {"$max":"$user.following_count"},
            "min_following_count": {"$min":"$user.following_count"},
            "avg_total_favorited": {"$avg":"$user.total_favorited"},
            "max_total_favorited": {"$max":"$user.total_favorited"},
            "min_total_favorited": {"$min":"$user.total_favorited"},
            "avg_dongtai_count": {"$avg":"$user.dongtai_count"},
            "max_dongtai_count": {"$max":"$user.dongtai_count"},
            "min_dongtai_count": {"$min":"$user.dongtai_count"}
        }
    }

pipeline = [match, group]

result = users.aggregate(pipeline)

for i in result:
    print(i)

##### 结论：

1. 明星的受关注度明显高于非明星，具体体现为follower更多，following更少，favorited更多
2. 网红并不包括在明星之中，理由是非明星也拥有十分可观的平均流量，这与现实情况并不匹配
3. 添加maxmin分析后发现，非明星中的大V，即网红，在流量表现上并不逊色于明星
4. 使用粉丝数大于100000进行筛选后，非明星的平均数据表现向明星逼近


##### 那么问题来了

Q:如何划定标准区分出网红？

A:【简单方案】粉丝数和受赞数混合筛选



In [14]:
import pandas as pd

df = pd.DataFrame(list(users.find()))

In [15]:
df_data = []
for user in users.find():
    df_data.append(user['user'])

In [16]:
df = pd.DataFrame(df_data)

In [17]:
df = df[['is_star', 'gender', 'birthday', 'follower_count', 'following_count', 'total_favorited', 'dongtai_count',
       'with_shop_entry', 'commerce_user_level', 'type_label']]

KeyError: "None of [Index(['is_star', 'gender', 'birthday', 'follower_count', 'following_count',\n       'total_favorited', 'dongtai_count', 'with_shop_entry',\n       'commerce_user_level', 'type_label'],\n      dtype='object')] are in the [columns]"

In [ ]:
df = df[df.is_star == False]
df = df[(df.birthday >= '1919-01-01') & (df.birthday <= '2013-01-01')]
df = df[df.follower_count < 10000]
df = df[df.total_favorited < 1000000]

In [ ]:
df.head()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

df.groupby([df.birthday]).is_star.count().plot(kind="bar")

plt.xlabel("")
ax=plt.gca()
counter = 0
for label in ax.get_xticklabels():
    if counter%4 != 0:
        label.set_visible(False)
    counter += 1
plt.show()

In [ ]:
df.groupby([df.gender]).is_star.count().plot(kind="bar")

plt.xlabel("")
ax=plt.gca()
plt.show()

In [ ]:
df.groupby([df.commerce_user_level]).is_star.count().plot(kind="bar")

plt.xlabel("")
ax=plt.gca()
plt.show()

In [ ]:
df_withcommerce = df[df.commerce_user_level == 1]

In [ ]:
df_withcommerce.head()

In [ ]:
df_withcommerce.groupby([df_withcommerce.birthday]).is_star.count().plot(kind="bar")

plt.xlabel("")
ax=plt.gca()
counter = 0
for label in ax.get_xticklabels():
    if counter%4 != 0:
        label.set_visible(False)
    counter += 1
plt.show()

In [ ]:
df_withcommerce.groupby([df_withcommerce.gender]).is_star.count().plot(kind="bar")
plt.show()

commerce_user_level等于1的用户从年龄分布上来看，大多数为80后90后，对应经济活动最频繁的一群人，猜想会和卖东西有关

In [ ]:
df.groupby([df.with_shop_entry]).is_star.count().plot(kind="bar")
plt.show()

现在回到全部数据对commerce和shop进行统计

In [ ]:
df = pd.DataFrame(df_data)
df = df[['is_star', 'gender', 'birthday', 'follower_count', 'following_count', 'total_favorited', 'dongtai_count',
       'with_shop_entry', 'commerce_user_level', 'type_label']]

In [ ]:
df.groupby([df.commerce_user_level]).is_star.count().plot(kind="bar")
plt.show()

In [ ]:
df.groupby([df.commerce_user_level]).is_star.count()

In [ ]:
df_withcommerce_1 = df[df.commerce_user_level == 1]
df_withcommerce_2 = df[df.commerce_user_level == 2]

In [ ]:
df_withcommerce_1.head()

In [ ]:
df_withcommerce_2.head()

In [ ]:
df_withcommerce_2.follower_count

In [ ]:
min(df_withcommerce_1.follower_count)

In [ ]:
min(df_withcommerce_2.follower_count)

In [ ]:
df_withcommerce_1.groupby([df_withcommerce_1.with_shop_entry]).is_star.count().plot(kind="bar")
plt.show()

In [ ]:
df_withcommerce_1.groupby([df_withcommerce_1.with_shop_entry]).is_star.count()

In [ ]:
df_withcommerce_2.groupby([df_withcommerce_2.with_shop_entry]).is_star.count()

In [ ]:
df_withshop = df[df.with_shop_entry == True]

In [ ]:
df_withshop.head()

In [ ]:
df_withcommerce_1.groupby([df_withcommerce_1.birthday]).is_star.count().plot(kind="bar")

ax=plt.gca()
counter = 0
for label in ax.get_xticklabels():
    if counter%4 != 0:
        label.set_visible(False)
    counter += 1
plt.show()

In [ ]:
df_withcommerce_2.groupby([df_withcommerce_2.birthday]).is_star.count().plot(kind="bar")

ax=plt.gca()
counter = 0
for label in ax.get_xticklabels():
    if counter%4 != 0:
        label.set_visible(False)
    counter += 1
plt.show()

In [ ]:
df_withshop.groupby([df_withshop.birthday]).is_star.count().plot(kind="bar")

ax=plt.gca()
counter = 0
for label in ax.get_xticklabels():
    if counter%4 != 0:
        label.set_visible(False)
    counter += 1
plt.show()

In [ ]:
df_withshop